In [40]:
!pip install -q gdown facenet-pytorch timm scikit-learn

import os

# Folder dataset
base_dir = "/content/dataset"
os.makedirs(base_dir, exist_ok=True)

# GDrive file id (ganti kalau beda)
file_id = "1tDo2zQC_1ZKY8aMYaalgr6nxRmBcRxaO"
zip_path = os.path.join(base_dir, "train_face_dataset.zip")

import gdown

if not os.path.exists(zip_path):
    print("📥 Downloading dataset...")
    gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_path, quiet=False)
else:
    print("✅ Zip dataset sudah ada.")

# Extract
extract_dir = base_dir
train_dir = os.path.join(extract_dir, "Train")

if not os.path.exists(train_dir):
    print("📂 Extracting...")
    import zipfile
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(extract_dir)
else:
    print("✅ Folder Train sudah ada.")

print("📁 Data dir:", train_dir)


📥 Downloading dataset...


Downloading...
From (original): https://drive.google.com/uc?id=1tDo2zQC_1ZKY8aMYaalgr6nxRmBcRxaO
From (redirected): https://drive.google.com/uc?id=1tDo2zQC_1ZKY8aMYaalgr6nxRmBcRxaO&confirm=t&uuid=13cc3cc0-9720-4a5d-a352-030fe31ba170
To: /content/dataset/train_face_dataset.zip
100%|██████████| 428M/428M [00:05<00:00, 73.6MB/s]

✅ Folder Train sudah ada.
📁 Data dir: /content/dataset/Train


In [41]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from facenet_pytorch import MTCNN
import timm

import numpy as np
from PIL import Image
from collections import Counter
from tqdm import tqdm
import os
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report,
)
from sklearn.linear_model import LogisticRegression

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥️ Device:", device)

# Path penting
data_dir = "/content/dataset/Train"

# Folder models
models_dir = os.path.join(os.getcwd(), "models")
os.makedirs(models_dir, exist_ok=True)
print("📁 models dir:", models_dir)


🖥️ Device: cuda
📁 models dir: /content/tubes-deep-learning/tubes-deep-learning/tubes-deep-learning/models


In [42]:
print("\n📸 Loading MTCNN...")
mtcnn = MTCNN(
    image_size=160,
    margin=20,
    device=device,
    keep_all=False,
    post_process=True,
)
print("✅ MTCNN loaded!")

def crop_face(image_path):
    """
    Baca gambar dari path, deteksi wajah dengan MTCNN,
    kembalikan face sebagai PIL.Image (atau None jika gagal).
    """
    try:
        img = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"⚠️ Gagal buka gambar {image_path}: {e}")
        return None

    with torch.no_grad():
        face = mtcnn(img)

    if face is None:
        return None

    # face.shape: (3, H, W) -> ubah ke PIL
    face = transforms.ToPILImage()(face)
    return face



📸 Loading MTCNN...
✅ MTCNN loaded!


In [43]:
# ==== DeiT-Small (Vision Transformer) ====
print("\n🧠 Loading DeiT-Small...")
vit = timm.create_model("deit_small_patch16_224", pretrained=True)
vit.reset_classifier(0)   # buang FC layer
vit.to(device)
vit.eval()
print("✅ DeiT-Small loaded!")

# Transform untuk DeiT
transform_vit = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

# ==== ResNet50 ====
print("\n🧠 Loading ResNet50 (ImageNet pretrained)...")
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
# buang FC -> hanya feature extractor
resnet.fc = nn.Identity()
resnet.to(device)
resnet.eval()
print("✅ ResNet50 loaded!")

# Transform untuk ResNet
transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

valid_ext = (".jpg", ".jpeg", ".png")



🧠 Loading DeiT-Small...
✅ DeiT-Small loaded!

🧠 Loading ResNet50 (ImageNet pretrained)...
✅ ResNet50 loaded!


In [44]:
from torchvision.transforms.functional import to_tensor

# ---------- Ekstraksi Embedding ----------

def extract_embedding_vit(face_pil):
    if face_pil is None:
        return None
    x = transform_vit(face_pil).unsqueeze(0).to(device)  # (1,3,224,224)
    with torch.no_grad():
        emb = vit(x)   # (1,384)
    emb = emb.view(-1).cpu().numpy()
    return emb  # (384,)


def extract_embedding_resnet(face_pil):
    if face_pil is None:
        return None
    x = transform_resnet(face_pil).unsqueeze(0).to(device)  # (1,3,224,224)
    with torch.no_grad():
        emb = resnet(x)   # (1,2048)
    emb = emb.view(-1).cpu().numpy()
    return emb  # (2048,)


# ---------- Build embeddings dari dataset ----------

def build_embeddings(dataset_dir, extractor_fn, desc="Building embeddings"):
    embeddings = []
    labels = []

    persons = [
        p for p in os.listdir(dataset_dir)
        if os.path.isdir(os.path.join(dataset_dir, p))
    ]

    for person_name in tqdm(persons, desc=desc):
        person_folder = os.path.join(dataset_dir, person_name)

        for img_name in os.listdir(person_folder):
            if not img_name.lower().endswith(valid_ext):
                continue

            img_path = os.path.join(person_folder, img_name)

            face_pil = crop_face(img_path)
            if face_pil is None:
                continue

            emb = extractor_fn(face_pil)
            if emb is None:
                continue

            embeddings.append(emb)
            labels.append(person_name)

    embeddings = np.array(embeddings)
    print(f"✅ {desc} selesai. Shape embeddings: {embeddings.shape}, labels: {len(labels)}")
    return embeddings, labels


# ---------- Evaluasi dengan Logistic Regression ----------

def train_eval_logreg(X, y, model_name, save_path):
    """
    Train-test split + LogisticRegression + classification report + save model.
    X: embeddings, y: label (list string)
    """
    # hitung test_size dinamis supaya > jumlah kelas
    freq = Counter(y)
    n_classes = len(freq)
    n_samples = len(y)
    # ambil ± 1/3 data untuk val tapi jaga > n_classes
    test_size = max(0.2, (n_classes + 1) / n_samples)
    test_size = min(test_size, 0.3)

    print(f"\n📌 {model_name}:")
    print("Total sampel:", n_samples)
    print("Total kelas :", n_classes)
    print("test_size   :", round(test_size, 3))

    X_train, X_val, y_train, y_val = train_test_split(
        X, y,
        test_size=test_size,
        random_state=42,
        stratify=y,
    )

    print("X_train:", X_train.shape, "X_val:", X_val.shape)

    clf = LogisticRegression(
        max_iter=500,
        multi_class="auto",
        solver="lbfgs",
        n_jobs=-1,
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_val)

    print("\n📊 Classification report:")
    print(classification_report(y_val, y_pred, zero_division=0))

    acc = accuracy_score(y_val, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_val, y_pred, average="weighted", zero_division=0
    )

    print(f"✅ Accuracy {model_name}: {acc:.4f}")
    print(f"✅ Weighted F1: {f1:.4f}")

    model_data = {
        "model_name": model_name,
        "classifier": clf,
        "train_embeddings": X_train,
        "train_labels": y_train,
        "val_labels": y_val,
        "accuracy": acc,
        "f1_weighted": f1,
    }

    with open(save_path, "wb") as f:
        pickle.dump(model_data, f)

    print(f"💾 Model {model_name} disimpan di: {save_path}")


In [45]:
print("\n🚀 Membuat embedding DeiT-Small...")
vit_embeddings, vit_labels = build_embeddings(
    data_dir, extract_embedding_vit, desc="DeiT-Small embeddings"
)

# Hapus kelas dengan sampel < 2
freq_vit = Counter(vit_labels)
print("\n📌 Jumlah sampel per kelas (ViT, sebelum filter):")
print(freq_vit)

vit_embeddings_filtered = []
vit_labels_filtered = []

for emb, lbl in zip(vit_embeddings, vit_labels):
    if freq_vit[lbl] >= 2:
        vit_embeddings_filtered.append(emb)
        vit_labels_filtered.append(lbl)

vit_embeddings_filtered = np.array(vit_embeddings_filtered)
print("\n✅ Setelah filter kelas < 2 sampel (ViT):")
print("Shape embeddings:", vit_embeddings_filtered.shape)
print("Total label:", len(vit_labels_filtered))

# Train + evaluasi LogReg
vit_model_path = os.path.join(models_dir, "face_recognition_deit_small_logreg.pkl")
train_eval_logreg(
    vit_embeddings_filtered,
    vit_labels_filtered,
    model_name="DeiT-Small + LogisticRegression",
    save_path=vit_model_path,
)



🚀 Membuat embedding DeiT-Small...


DeiT-Small embeddings:  81%|████████▏ | 57/70 [01:38<00:30,  2.33s/it]

⚠️ Gagal buka gambar /content/dataset/Train/Martua Kevin A.M.H.Lubis/IMG_20241208_223409_653 - MARTUA KEVIN ANDREAS MUAL H LUBIS.jpg: cannot identify image file '/content/dataset/Train/Martua Kevin A.M.H.Lubis/IMG_20241208_223409_653 - MARTUA KEVIN ANDREAS MUAL H LUBIS.jpg'


DeiT-Small embeddings:  91%|█████████▏| 64/70 [01:48<00:09,  1.57s/it]

⚠️ Gagal buka gambar /content/dataset/Train/Lois Novel E Gurning/foto 3 - LOIS NOVEL E. GURNING.png: cannot identify image file '/content/dataset/Train/Lois Novel E Gurning/foto 3 - LOIS NOVEL E. GURNING.png'
⚠️ Gagal buka gambar /content/dataset/Train/Lois Novel E Gurning/foto 4 - LOIS NOVEL E. GURNING.png: cannot identify image file '/content/dataset/Train/Lois Novel E Gurning/foto 4 - LOIS NOVEL E. GURNING.png'


DeiT-Small embeddings: 100%|██████████| 70/70 [01:57<00:00,  1.68s/it]

✅ DeiT-Small embeddings selesai. Shape embeddings: (243, 384), labels: 243

📌 Jumlah sampel per kelas (ViT, sebelum filter):
Counter({'Joshua Palti Sinaga': 5, 'Bezalel Samuel Manik': 4, 'hayyatul fajri': 4, 'Arkan Hariz Chandrawinata Liem': 4, 'Fajrul Ramadhana Aqsa': 4, 'Kenneth Austin Wijaya': 4, 'Zefanya Danovanta Tarigan': 4, 'JP. Rafi Radiktya Arkan. R. AZ': 4, 'Dyo Dwi Carol Bukit': 4, 'Ikhsannudin Lathief': 4, 'Joyapul Hanscalvin Panjaitan': 4, 'Fayyadh Abdillah': 4, 'Gabriella Natalya Rumapea': 4, 'Dwi Arthur Revangga': 4, 'Zaky Ahmad Makarim': 4, 'Royfran Roger Valentino': 4, 'Desty Ananta Purba': 4, 'Abu Bakar Siddiq Siregar': 4, 'Femmy Aprillia Putri': 4, 'Bayu Ega Ferdana': 4, 'Rahmat Aldi Nasda': 4, 'Ichsan Kuntadi Baskara': 4, 'Falih Dzakwan Zuhdi': 4, 'Randy Hendriyawan': 4, 'Joy Daniella V': 4, 'Garland Wijaya': 4, 'Reynaldi Cristian Simamora': 4, 'Zakhi algifari': 4, 'Ahmad Faqih Hasani': 4, 'Dimas Azi Rajab Aizar': 4, 'Freddy Harahap': 4, 'Alfajar': 4, 'Rayhan Fatih 


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



📊 Classification report:
                                 precision    recall  f1-score   support

             Nasya Aulia Efendi       0.00      0.00      0.00         1
           Abraham Ganda Napitu       0.00      0.00      0.00         1
       Abu Bakar Siddiq Siregar       1.00      1.00      1.00         1
             Ahmad Faqih Hasani       0.00      0.00      0.00         1
                        Alfajar       1.00      1.00      1.00         1
            Alief Fathur Rahman       0.00      0.00      0.00         1
 Arkan Hariz Chandrawinata Liem       1.00      1.00      1.00         1
               Bayu Ega Ferdana       0.00      0.00      0.00         1
          Bayu Prameswara Haris       0.00      0.00      0.00         1
           Bezalel Samuel Manik       0.50      1.00      0.67         1
           Bintang Fikri Fauzan       0.00      0.00      0.00         1
              Boy Sandro Sigiro       0.00      0.00      0.00         1
             Desty Anant

In [46]:
print("\n🚀 Membuat embedding ResNet50...")
res_embeddings, res_labels = build_embeddings(
    data_dir, extract_embedding_resnet, desc="ResNet50 embeddings"
)

freq_res = Counter(res_labels)
print("\n📌 Jumlah sampel per kelas (ResNet, sebelum filter):")
print(freq_res)

res_embeddings_filtered = []
res_labels_filtered = []

for emb, lbl in zip(res_embeddings, res_labels):
    if freq_res[lbl] >= 2:
        res_embeddings_filtered.append(emb)
        res_labels_filtered.append(lbl)

res_embeddings_filtered = np.array(res_embeddings_filtered)
print("\n✅ Setelah filter kelas < 2 sampel (ResNet):")
print("Shape embeddings:", res_embeddings_filtered.shape)
print("Total label:", len(res_labels_filtered))

# Train + evaluasi LogReg
res_model_path = os.path.join(models_dir, "face_recognition_resnet50_logreg.pkl")
train_eval_logreg(
    res_embeddings_filtered,
    res_labels_filtered,
    model_name="ResNet50 + LogisticRegression",
    save_path=res_model_path,
)



🚀 Membuat embedding ResNet50...


ResNet50 embeddings:  81%|████████▏ | 57/70 [01:42<00:33,  2.59s/it]

⚠️ Gagal buka gambar /content/dataset/Train/Martua Kevin A.M.H.Lubis/IMG_20241208_223409_653 - MARTUA KEVIN ANDREAS MUAL H LUBIS.jpg: cannot identify image file '/content/dataset/Train/Martua Kevin A.M.H.Lubis/IMG_20241208_223409_653 - MARTUA KEVIN ANDREAS MUAL H LUBIS.jpg'


ResNet50 embeddings:  91%|█████████▏| 64/70 [01:53<00:09,  1.66s/it]

⚠️ Gagal buka gambar /content/dataset/Train/Lois Novel E Gurning/foto 3 - LOIS NOVEL E. GURNING.png: cannot identify image file '/content/dataset/Train/Lois Novel E Gurning/foto 3 - LOIS NOVEL E. GURNING.png'
⚠️ Gagal buka gambar /content/dataset/Train/Lois Novel E Gurning/foto 4 - LOIS NOVEL E. GURNING.png: cannot identify image file '/content/dataset/Train/Lois Novel E Gurning/foto 4 - LOIS NOVEL E. GURNING.png'


ResNet50 embeddings: 100%|██████████| 70/70 [02:03<00:00,  1.76s/it]
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


✅ ResNet50 embeddings selesai. Shape embeddings: (243, 2048), labels: 243

📌 Jumlah sampel per kelas (ResNet, sebelum filter):
Counter({'Joshua Palti Sinaga': 5, 'Bezalel Samuel Manik': 4, 'hayyatul fajri': 4, 'Arkan Hariz Chandrawinata Liem': 4, 'Fajrul Ramadhana Aqsa': 4, 'Kenneth Austin Wijaya': 4, 'Zefanya Danovanta Tarigan': 4, 'JP. Rafi Radiktya Arkan. R. AZ': 4, 'Dyo Dwi Carol Bukit': 4, 'Ikhsannudin Lathief': 4, 'Joyapul Hanscalvin Panjaitan': 4, 'Fayyadh Abdillah': 4, 'Gabriella Natalya Rumapea': 4, 'Dwi Arthur Revangga': 4, 'Zaky Ahmad Makarim': 4, 'Royfran Roger Valentino': 4, 'Desty Ananta Purba': 4, 'Abu Bakar Siddiq Siregar': 4, 'Femmy Aprillia Putri': 4, 'Bayu Ega Ferdana': 4, 'Rahmat Aldi Nasda': 4, 'Ichsan Kuntadi Baskara': 4, 'Falih Dzakwan Zuhdi': 4, 'Randy Hendriyawan': 4, 'Joy Daniella V': 4, 'Garland Wijaya': 4, 'Reynaldi Cristian Simamora': 4, 'Zakhi algifari': 4, 'Ahmad Faqih Hasani': 4, 'Dimas Azi Rajab Aizar': 4, 'Freddy Harahap': 4, 'Alfajar': 4, 'Rayhan Fati